In [ ]:
import os
import glob
import mne
from mne import Report
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import plotly.graph_objects as go
from utils import (get_parent_dir, extract_lab_sec, loc_of_sep_lines, plot_seizure_horizontal, plot_seizure_vertical,
                                        raw_to_df, extract_groups, extract_ordered_groups, save_plotly_to_html,
                                        shrink_df_to_tmax, create_results_folders, save_fig_to_disc, plot_interactive_subplot_with_table,
                                        extract_parameters_from_raw, plot_eventcounts, plot_interactive_tables)
import plotly as py
import ipywidgets as widgets
import plotly.io as pio
from plotly.subplots import make_subplots

py.offline.init_notebook_mode(connected=True)
graph_sep_line_width = 5
plot_tmin = 5
plot_tmax = 60

In [ ]:
def extract_parameters_from_raw(raw=None):
    highp = raw.info["highpass"]
    lowp = raw.info["lowpass"]
    sfreq = raw.info["sfreq"]
    aq = raw.info["meas_date"]
    channels = raw.info["ch_names"]
    nr_channels = raw.info["nchan"]
    return highp, lowp, sfreq, aq, channels, nr_channels

def raw_to_df(raw, edf=None):
    df = pd.DataFrame(raw.annotations)
    to_drop = ["duration"]
    df = df.drop(to_drop, axis=1)
    if "Beginn" in str(df["description"]):
        samp_beginn = df[df["description"].str.contains("Beginn")]["onset"]
        onset = samp_beginn
        if isinstance(samp_beginn, pd.core.series.Series):
            print(f"There are multiple markers for seizure onset in this file --> taking first one.")
            samp_beginn = samp_beginn.iloc[0]
            onset = samp_beginn
    else:
        print("Error: No marker containing \"Beginn\" found, cannot determine seizure onset for file: ", edf)
        print("Setting seizure onset to the beginning of the file")
        samp_beginn = 0
        onset = "No seizure onset was marked"
    df["time_from_onset"] = df["onset"] - float(samp_beginn)
    df = df.drop(["orig_time"], axis=1)
    return df, onset

In [ ]:
edfs = glob.glob("../data/*.edf")
e = edfs[0]
e

In [ ]:
raw = mne.io.read_raw(e)
df, onset = raw_to_df(raw, e)
e_events, s_events, t_events = extract_groups(df, e)
e_events

In [ ]:
def plot_eventcounts(df=None, eeg=None, semio=None, source=None):
    fig, ax = plt.subplots(figsize=(15,26))
    plt.suptitle(str(source) + " - Event counts")
    #EEG
    ax1 = fig.add_subplot(2, 1, 1)
    ax1.set_title("EEG Events")
    if len(eeg['description'].value_counts()) > 0:
        sns.countplot(y="description", data=eeg, orient="h", ax=ax1, order = eeg['description'].value_counts().index)
    else:
        #plt.plot(x=1, y=1)
        ax1.set_title("No EEG events found in file...")
    #Semiology
    ax2 = fig.add_subplot(2, 1, 2)
    ax2.set_title("Semiology Events")
    if [semio['description'].value_counts()] != []:
        sns.countplot(y="description", data=semio, orient="h", ax=ax2, order = semio['description'].value_counts().index)
    else:
        plt.plot(x=1, y=1)
        ax2.set_title("No Semiology events found in file...")
    ax.set_yticks([])
    ax.set_axis_off()
    fig.subplots_adjust(hspace=0.2)
    return fig

fig = plot_eventcounts(df=df, eeg=e_events, semio=s_events, source=e)

In [ ]:
df.to_csv("df_template.csv")

In [ ]:
df = pd.read_csv("df_template.csv")
e_events, s_events, t_events = extract_groups(df, "Test_Dataframe")

In [ ]:
fig = plot_eventcounts(df=df, eeg=e_events, semio=s_events, source=e)

In [ ]:
import plotly.express as px
def plot_interactive_eventcounts(eeg=None, semio=None, test=None, source=None):
    fig = make_subplots(rows=3, cols=1, 
                        specs=[[{"type": "bar"}],
                                [{"type": "bar"}],
                                [{"type": "bar"}]],
                        subplot_titles=("EEG events", "Semiology events", "Testing events"),
                        #row_width=[0.1, 0.1, 0.1],
                        vertical_spacing=0.2
                        )
    # EEG
    fig.add_trace(go.Histogram(y=e_events["description"], 
                        histfunc="count",
                        orientation="h",
                        name="EEG"),
                    row=1, col=1
                    )

    # Semio
    fig.add_trace(go.Histogram(y=s_events["description"], 
                        histfunc="count",
                        orientation="h",
                        name="Semiology"),
                    row=2, col=1
                    )    

    # Test
    fig.add_trace(go.Histogram(y=t_events["description"], 
                        histfunc="count",
                        orientation="h",
                        name="Testing"),
                    row=3, col=1
                    )


    fig.update_yaxes(categoryorder="total descending")
    fig.update_layout(width=1000, height=1200)
    return fig

plot_interactive_eventcounts(eeg=e_events, semio=s_events, test=t_events, source="Test")

In [ ]:
def plot_interactive_eeg_and_semio(eeg=None, semio=None, test=None, source=None):
    fig = make_subplots(rows=1, cols=2, start_cell="top-left",
                        subplot_titles=("EEG events", "Semiology events"),
                        #row_width=[0.1, 0.1, 0.1],
                        horizontal_spacing=0.2
                        )
    # EEG
    fig.add_trace(go.Histogram(y=e_events["description"], 
                        histfunc="count",
                        orientation="h",
                        name="EEG"),
                    row=1, col=1
                    )

    # Semio
    fig.add_trace(go.Histogram(y=s_events["description"], 
                        histfunc="count",
                        orientation="h",
                        name="Semiology"),
                    row=1, col=2
                    )    


    fig.update_yaxes(categoryorder="total descending")
    fig.update_layout(width=1100, height=800)
    return fig

plot_interactive_eventcounts(eeg=e_events, semio=s_events, test=t_events, source="Test")

In [ ]:
t_events

In [ ]:
def plot_interactive_eventcount(df=None, mode=None, source=None):
    fig = go.Figure(
        data=[go.Histogram(y=df["description"], 
                            histfunc="count",
                            orientation="h")]
                    )
    fig.update_yaxes(categoryorder="total descending")
    fig.update_layout(title=(source + " - " + mode + " - Eventcount"))
    return fig
plot_interactive_eventcount(t_events, mode="Semiology", source= "asd")

In [ ]:
t_events.loc[:,"failed"]=0
succeeded = "1" in t_events["description"]
t_events.loc[t_events, "failed"] = 1

fig = px.scatter(t_events, x=t_events["onset"], y=t_events["description"], color="failed",
                 title="Testing results",
                 labels={"salary":"Annual Salary (in thousands)"} # customize axis label
                )

fig.show()

In [ ]:
t_events["description"]

In [ ]:
"""
for testing 
--> color = "gender" wird zu 1 oder 0 (korrekt gemacht oder nicht)
--> x Achse wird Zeit
--> y Achse wird s_events["description"]

"""

import plotly.express as px
import pandas as pd

schools = ["Brown", "NYU", "Notre Dame", "Cornell", "Tufts", "Yale",
           "Dartmouth", "Chicago", "Columbia", "Duke", "Georgetown",
           "Princeton", "U.Penn", "Stanford", "MIT", "Harvard"]
n_schools = len(schools)

men_salary = [72, 67, 73, 80, 76, 79, 84, 78, 86, 93, 94, 90, 92, 96, 94, 112]
women_salary = [92, 94, 100, 107, 112, 114, 114, 118, 119, 124, 131, 137, 141, 151, 152, 165]

df = pd.DataFrame(dict(school=schools*2, salary=men_salary + women_salary,
                       gender=["Men"]*n_schools + ["Women"]*n_schools))

# Use column names of df for the different parameters x, y, color, ...
fig = px.scatter(df, x="salary", y="school", color="gender",
                 title="Gender Earnings Disparity",
                 labels={"salary":"Annual Salary (in thousands)"} # customize axis label
                )

fig.show()

